### This purpose of this code is to read raw altimetry data from a server in Alaska and then ingest it into the postgres database

#### we'll use the Python library http://pysftp.readthedocs.org/en/release_0.2.8/

#### A. Arendt, S. Nimmagadda, C. Larsen

##### rev. 2015 11 17: initial creation
##### 2015 11 29: stepping through ReadLambFile from Evan's UpdateDb.py; removing str/dict distinction; trying in Pandas instead

establish the secure FTP connection:

In [1]:
%matplotlib inline 
import pandas as pd
import pysftp
import Altimetry as alt
import UpdateDb as udb
from base64 import b64decode as readpassword
from sqlalchemy import create_engine

In [10]:
user = 'sai'
word = 'dGVzdA=='

engine = create_engine('postgresql://' + user + ':' + readpassword(word) + '@localhost:5432/sandbox')

In [5]:
# just do this once
engine.execute("CREATE TABLE lambtest (gid serial PRIMARY KEY, glid smallint, date1 date, date2 date, interval smallint, volmodel real, vol25diff real,vol75diff real, balmodel real, bal25diff real,bal75diff real,e integer[],dz real[],dz25 real[],dz75 real[],aad real[],masschange real[],massbal real[],numdata integer[]);")

In [6]:
host = readpassword('YmFpcmQuZ3BzLmFsYXNrYS5lZHU=')
user = readpassword('bGFzZXI=')
pw = readpassword('MjYpKSlrbQ==')
sftp = pysftp.Connection(host, username=user, password=pw)

with sftp.cd('/home/laser/analysis/'):
    glacierList = sftp.listdir()
    for glacierName in glacierList:
        print(glacierName)
        if sftp.exists(glacierName + '/results'):
            with (sftp.cd(glacierName + '/results/')):
                for fileName in sftp.listdir():
                    if fileName.endswith(".output.txt"):
                        sftp.get(fileName)
                        sql = udb.import_lamb_file_to_db(fileName,'lambtest')
                        engine.execute(sql)

In [ ]:
import Altimetry as alt
%matplotlib inline
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
surveyeddata = alt.GetLambData(verbose=False,longest_interval=True,interval_max=30,interval_min=5,by_column=True,
                           as_object=True)
surveyeddata.fix_terminus()
surveyeddata.normalize_elevation()
surveyeddata.calc_dz_stats()

#Now to partition the dataset as done in Larsen etal by glacier type.
#We excluding surgers and outlier glaciers because we don't want those glaciers to affect 
#   mean profiles used for extrapolation.
types = ["gltype=0","gltype=1","gltype=2"]
lamb,userwheres,notused,whereswo,notswo = alt.partition_dataset(types,applytoall=["surge='f'","name NOT IN ('Columbia Glacier','West Yakutat Glacier','East Yakutat Glacier')"])
results = alt.extrapolate('testing',lamb,whereswo,insert_surveyed_data=surveyeddata,keep_postgres_tbls=False) 

In [ ]:
import numpy as N
#Plotting a pie chart of the total amount of mass lost from each glacier type.
typenames = {'0':'Land','1':'Tidewater','2':'Lake'}
labels = [typenames[i] for i in results['bytype']['gltype'].astype(str)]
figure(figsize=(3,3))
pi = pie(N.abs(results['bytype']['totalgt']),labels=labels)

In [ ]:
results['bytype']['totalgt'].sum()